In [89]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
from tensorflow import keras

In [90]:
!git clone https://github.com/Juandi11/Series_de_tiempo.git #Se lee la URL donde se alojaron las series de tiempo

Cloning into 'Series_de_tiempo'...
remote: Enumerating objects: 10004, done.
remote: Total 10004 (delta 0), reused 0 (delta 0), pack-reused 10004
Receiving objects: 100% (10004/10004), 1.35 GiB | 14.74 MiB/s, done.
Updating files: 100% (10001/10001), done.


In [111]:
malos = [156, 751, 947, 1314, 1586, 1739, 1742, 2075, 2286, 2528, 2747, 2939, 2961, 3030, 3226, 3245, 4276,
         4357, 4724, 4725, 4799, 4855, 5154, 5224, 5281, 5370, 5497, 5577, 5609, 5717, 5740, 5945, 6049,
         6366, 7607, 7754, 7909, 7950, 8099, 8307, 8331, 8556, 8826, 9038, 9276, 9510, 9984] #series de tiempo con datos erróneos

cantidad=3200 #Cantidad de series de tiempo utilizadas para Train y Test

Apredecir = [] # aqui van las series utilizadas para Train y Test, excluyendo las series de tiempo malas o con datos erróneos
for i in range(1, cantidad+1):
  if (not i in malos): # se excluyen las que están malas
    Apredecir.append(i) #se agregan a la lista vacía

s = [] # aqui se guardan las series ya procesadas

tiempo_05= 62 # ventana de 0.5 segundos
tiempo2 = 242 # ventana de 2 segundos
tiempo4 = 482 # ventana de 4 segundos
tiempo5 = 602 # ventana de 5 segundos
tiempo6 = 722 # ventana de 6 segundos

for i in tqdm(Apredecir):
  ser = pd.read_csv(f'/content/Series_de_tiempo/tseries_CSV/tseries{i}.csv', sep = ',') # se leen los archivos del github
  ser = ser.iloc[1:tiempo2,1:].apply(pd.to_numeric).to_numpy() # se toman solo los valores numericos indicados en la ventana de tiempo dada
  s.append(ser) # se añade a la lista de series
ser = np.array(s) # se pasa a un array el total de series
del(s) #Se elimina la lista s

# se lee el archivo de etiquetas de ceros (0s) y unos (1s) de las series a usar en Train y Test
y = pd.read_excel('/content/Series_de_tiempo/tseries_CSV/Lyapunov_analysis_10000_cases_mios_clasif.xlsx')['Lyapunov'][Apredecir].to_numpy(float)


100%|██████████| 3186/3186 [02:05<00:00, 25.40it/s]


In [112]:
ser.shape

(3186, 241, 39)

In [113]:
y.shape

(3186,)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(ser, y, test_size=0.2, random_state=42) #Se parte el conjunto de datos en Train y Test

In [115]:
X_train.shape

(2548, 241, 39)

In [116]:
y_train.shape

(2548,)

In [117]:
X_test.shape

(638, 241, 39)

In [118]:
sum(y_train)

419.0

In [119]:
#Sección de balanceo de datos
s1,s2,s3 = X_train.shape[0],X_train.shape[1],X_train.shape[2]
print(s1,s2,s3)
X_train = np.reshape(X_train,(s1, s2*s3)) #Se hace una redimensión de los datos para entrarlos al proceso de balanceo
print(X_train.shape)

#Importo nueva librería
from imblearn.combine import SMOTEENN

smenn=SMOTEENN()

# Balanceo de datos
X_resampled, y_train = smenn.fit_resample(X_train, y_train)

#Se vuelve a las dimensiones originales de los datos de Train
X_resampled = np.reshape(X_resampled, (X_resampled.shape[0], s2, s3))


2548 241 39
(2548, 9399)


In [120]:
X_resampled.shape

(3875, 241, 39)

In [121]:
y_train.shape

(3875,)

In [122]:
sum(y_train)

1981.0

In [123]:
#Se crea el modelo
model = Sequential([
    LSTM(35, input_shape=(ser.shape[1], ser.shape[2]), return_sequences=True),
    LSTM(35),
    Dense(1, activation='sigmoid')
])
#Métricas a utilizar
Metrics = [
      keras.metrics.BinaryAccuracy(name='Exactitud'),
      keras.metrics.Precision(name='Precisión'),
      keras.metrics.Recall(name='Recall'),
      ]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=Metrics)

In [124]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 241, 35)           10500     
                                                                 
 lstm_13 (LSTM)              (None, 35)                9940      
                                                                 
 dense_6 (Dense)             (None, 1)                 36        
                                                                 
Total params: 20476 (79.98 KB)
Trainable params: 20476 (79.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [125]:
# Entrenamiento del modelo
model.fit(X_resampled, y_train, epochs=20)

Epoch 1/20
122/122 [==============================] - 29s 195ms/step - loss: 0.4240 - Exactitud: 0.8196 - Precisión: 0.9319 - Recall: 0.6981
Epoch 2/20
122/122 [==============================] - 25s 207ms/step - loss: 0.3100 - Exactitud: 0.8795 - Precisión: 0.9725 - Recall: 0.7865
Epoch 3/20
122/122 [==============================] - 25s 206ms/step - loss: 0.2891 - Exactitud: 0.8805 - Precisión: 0.9651 - Recall: 0.7951
Epoch 4/20
122/122 [==============================] - 25s 203ms/step - loss: 0.2258 - Exactitud: 0.9133 - Precisión: 0.9768 - Recall: 0.8506
Epoch 5/20
122/122 [==============================] - 24s 196ms/step - loss: 0.6298 - Exactitud: 0.6867 - Precisión: 0.7314 - Recall: 0.6118
Epoch 6/20
122/122 [==============================] - 27s 223ms/step - loss: 0.3166 - Exactitud: 0.8699 - Precisión: 0.9701 - Recall: 0.7693
Epoch 7/20
122/122 [==============================] - 25s 207ms/step - loss: 0.5310 - Exactitud: 0.7368 - Precisión: 0.7914 - Recall: 0.6588
Epoch 8/20
12

In [126]:
# Prueba de Test
predictions = model.predict(X_test)
p = predictions > .5 # los que tienen probabilidad mayor a 50% de ser 1 se ponen como 1, el resto es 0
r = y_test.reshape(y_test.shape[0], 1)
print(100*np.mean(p == r)) # promedio de porcentaje de predicciones correctas

20/20 [==============================] - 2s 46ms/step
92.94670846394985


In [127]:
X_test.shape

(638, 241, 39)

In [128]:
y_test.shape

(638,)

In [129]:
#Resultados de Test
print(confusion_matrix(y_test, p.astype(int)))
report = classification_report(y_test, p.astype(int))
print("\nInforme de clasificación:")
print(report)

[[497  25]
 [ 20  96]]

Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       522
         1.0       0.79      0.83      0.81       116

    accuracy                           0.93       638
   macro avg       0.88      0.89      0.88       638
weighted avg       0.93      0.93      0.93       638



In [130]:
#Lectura de datos para validación
cantidad_val1=cantidad+1
cantidad_val2=cantidad+300 #Se utilizan otras 300 series de tiempo más para la validación

Avalidacion = [] # aqui van las series que se van a utilizar para validación
for i in range(cantidad_val1, cantidad_val2+1):
  if (not i in malos): # se excluyen las que están malas
    Avalidacion.append(i) #Se agregan a la lista vacía

s2 = [] # aqui se guardan las series ya procesadas

for i in tqdm(Avalidacion):
  ser2 = pd.read_csv(f'/content/Series_de_tiempo/tseries_CSV/tseries{i}.csv', sep = ',') # se leen los archivos del github
  ser2 = ser2.iloc[1:tiempo2,1:].apply(pd.to_numeric).to_numpy() # se toman solo los valores numericos indicados en la ventan de tiempo dada
  s2.append(ser2) # se añade a la lista de series
ser2 = np.array(s2) # se pasa a un array el total de series
del(s2) #se elimina la lista s2

# se lee el archivo de etiquetas de ceros (0s) y unos (1s) de las series a usar para Validación
y2 = pd.read_excel('/content/Series_de_tiempo/tseries_CSV/Lyapunov_analysis_10000_cases_mios_clasif.xlsx')['Lyapunov'][Avalidacion].to_numpy(float)

100%|██████████| 298/298 [00:11<00:00, 25.40it/s]


In [131]:
ser2.shape

(298, 241, 39)

In [132]:
# Prueba de Validación
predictions_valid = model.predict(ser2)
p2 = predictions_valid > .5 # los que tienen probabilidad mayor a 50% de ser 1 se ponen como 1, el resto es 0
r2 = y2.reshape(y2.shape[0], 1)
print(100*np.mean(p2 == r2)) # promedio de porcentaje de predicciones correctas

10/10 [==============================] - 0s 45ms/step
94.29530201342283


In [133]:
#Resultados de Validación
print(confusion_matrix(y2, p2.astype(int)))
report2 = classification_report(y2, p2.astype(int))
print("\nInforme de clasificación:")
print(report2)

[[248   6]
 [ 11  33]]

Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97       254
         1.0       0.85      0.75      0.80        44

    accuracy                           0.94       298
   macro avg       0.90      0.86      0.88       298
weighted avg       0.94      0.94      0.94       298

